In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pickle
import logging

In [3]:
# Set logging level to INFO
logging.basicConfig(level=logging.INFO)

In [4]:
class Message:
    ''' Class to keep sender, timestamp, and public/hidden recipients of a message
    
    Must map the Message class fields in parseMaildir.py for deserialization purposes
    '''
    def __init__(self, From, mtime, To, Cc, Bcc):
        self.From = From
        self.mtime = mtime
        self.To = To
        self.Cc = Cc
        self.Bcc = Bcc

In [5]:
'''Parse the pickle files generated by parseMaildir.py'''
parsedLogsFolder = 'Enron/parsing/'
social_graph = pickle.load(open(parsedLogsFolder + "social.pkl", "rb"))
log = pickle.load(open(parsedLogsFolder + "replay_log.pkl", "rb"))

In [6]:
def prep_static_view(social_graph):
    ''' Prepare the userset and chain_head_dict structs for the static view scenario
    
    In the static view scenario, all users update their encryption key at the beginning of time. We then
    simulate how the updated keys get propagated after replaying all messages in chronological order.
    We instantiate the `userset` and `chain_head_dict` structs as follows:
    
      * We include in `userset` the users that we have full access to their sent messages
      * We boostrap the social graph of each user in the userset with her future recipients
      * We create a `chain_head_dict` struct to keep track of the latest known head of a user's friends
    '''
    
    # Set of users we know the social graph for
    userset = set([])

    # Dictionary to keep track of the latest head known of a user's friends
    chain_head_dict = {}

    # Initialize the latest known head dictionary
    for user in social_graph:
        userset.add(user['from_header'])
        chain_head_dict[user['from_header'], user['from_header']] = 1
        for friend in user['friends']:
            chain_head_dict[(user['from_header'], friend)] = 0

    return userset, chain_head_dict

In [7]:
def sim_priv_capabilities(userset, chain_head_dict):
    for email in log:
        if email.From not in userset:
            continue

        for recipient in email.To | email.Cc | email.Bcc - {email.From}:
            # For all recipients, update the dict entry for the sender
            chain_head_dict[(recipient, email.From)] = chain_head_dict[email.From, email.From]
            logging.debug("User %s updated the head for user %s" % (recipient, email.From))

            # For all recipients, update the dict entries for the other public recipients
            for other in email.To | email.Cc - {recipient}:
                if (email.From, other) in chain_head_dict:
                    if not (recipient, other) in chain_head_dict:
                        chain_head_dict[(recipient, other)] = chain_head_dict[(email.From, other)]
                        logging.debug("User %s updated the head for user %s" % (recipient, other))

    return chain_head_dict

In [14]:
def eval_head_propagation(chain_head_dict):
    updated = 0
    stale = 0
    not_updated = 0
    for user, friend in chain_head_dict:
        if user == friend or user not in userset:
            continue
        if chain_head_dict[(user, friend)] == 0:
            not_updated += 1
            continue
        if (friend, friend) in chain_head_dict:
            if chain_head_dict[(user, friend)] == chain_head_dict[(friend, friend)]:
                updated += 1
            else:
                stale += 1
        else:
            if chain_head_dict[(user, friend)] == 1:
                updated += 1

    return updated, stale, not_updated

In [15]:
print("Simulate the static view scenario for Private Capabilities:")
userset, chain_head_dict = prep_static_view(social_graph)
chain_head_dict = sim_priv_capabilities(userset, chain_head_dict)
updated, stale, not_updated = eval_head_propagation(chain_head_dict)
print("There were %s updates on the latest head dictionary, but %s entries are not updated." % (updated + stale, not_updated))

Simulate the static view scenario for Private Capabilities:
There were 475 updates on the latest head dictionary, but 13342 entries are not updated.
